In [107]:
temp = np.zeros(shape=(n_vertex, n_vertex))
np.fill_diagonal(temp, A)

In [106]:
A = A.reshape(-1,1)
A.shape

(6449, 1)

In [1]:
csr_matrix((A, (dia_idx, dia_idx)), shape=(n_vertex, n_vertex), dtype=dtype)

NameError: name 'csr_matrix' is not defined

In [79]:
# dtype = np.float32
# n_eigs = 50

# n_vertex = random_meshes[0].num_vertices
# all_evecs = np.zeros(shape=(n_meshes, n_vertex, n_eigs), dtype=dtype)
# all_evals = np.zeros(shape=(n_meshes, n_eigs), dtype=dtype)

# for i, in_m in enumerate(random_meshes):
#     lb = Laplace_Beltrami(in_m)
#     all_evals[i], all_evecs[i] = lb.spectra(n_eigs)
    
all_areas = []
# dia_idx = np.arange(n_vertex)
for i, in_m in enumerate(random_meshes):    
    A = in_m.area_of_vertices().reshape(-1)
    temp = np.zeros(shape=(n_vertex, n_vertex))
    
    all_areas
#     all_areas.append(csr_matrix((A, (dia_idx, dia_idx)), shape=(n_vertex, n_vertex), dtype=dtype))
#     all_areas[-1] = all_areas[-1].toarray() # MAKE-dense

KeyboardInterrupt: 

In [26]:
n_edges = 200
gg = gnm_random_graph(n_meshes, n_edges)
is_directed = False
g = Graph(gg, is_directed)

In [28]:
s, t = g.edges()

In [67]:
all_fmaps = []
for i, j in zip(s, t):
    all_fmaps.append(all_evecs[j].T.dot(all_areas[j]).dot(all_evecs[i]))

KeyboardInterrupt: 

In [67]:
Graph.connected_components(g)

(1, array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0], dtype=int32))

In [46]:
n = 100
g = gnm_random_graph(n, 10, seed=None, directed=False)

In [30]:
b = coo_matrix((vals, (source_e, target_e)), shape=(n, n)).toarray()

In [32]:
np.all(g == b)

True

In [162]:
# splits = prepare_train_test_val(n_shapes, in_labels, train_per, test_per, seed=seed)

# if use_pc: 
#     feeds = in_pcs
# else:
#     feeds = in_diffs

# in_data = make_data(splits, feeds, in_labels)

from tf_lab.data_sets.numpy_dataset import NumpyDataset
def make_data_mixed(in_data, in_pc, class_labels, in_diff):
    res = dict()
    for s in ['train', 'test', 'val']:
        idx = in_data[s].copy()        
        res[s] = NumpyDataset([in_pc[idx], class_labels[idx], in_diff[idx]], ['feed', 'labels', 'ids'])
    return res

in_data = make_data_mixed(splits, in_pcs, in_labels, in_diffs)

['feed', 'labels', 'ids']
['feed', 'labels', 'ids']
['feed', 'labels', 'ids']


In [123]:
use_pc = False
arch = 'conv'

In [176]:
reset_tf_graph()

if use_pc: 
    with tf.variable_scope('pc_based_net') as scope:
        feed_pl = tf.placeholder(tf.float32, shape = (None, n_pc_points, 3) )
        labels_pl = tf.placeholder(tf.int64, shape=[None])                           
        layer = encoder_with_convs_and_symmetry_new(feed_pl, n_filters=[64, 128], b_norm=False)
        layer = fully_connected(layer, 64, activation='relu', weights_init='xavier')
        logits = fully_connected(layer, n_classes, activation='linear', weights_init='xavier')
else:
    labels_pl = tf.placeholder(tf.int64, shape=[None])
    diff_dims = (60, 30)
    feed_pl_1 = tf.placeholder(tf.float32, shape = (None, n_pc_points, 3) )
    feed_pl_2 = tf.placeholder(tf.float32, shape = (None, ) + diff_dims)
    
    layer_pc = encoder_with_convs_and_symmetry_new(feed_pl_1, n_filters=[64, 128], b_norm=False)
    
    
    layer = conv_2d(tf.expand_dims(feed_pl_2, -1), nb_filter=4, filter_size=4, activation='relu')
    layer = max_pool_2d(layer, kernel_size=2)
    layer = conv_2d(layer, nb_filter=4, filter_size=3, activation='relu')
    layer = max_pool_2d(layer, kernel_size=2)
    layer = fully_connected(layer, 6, activation='relu', weights_init='xavier')    
    layer_diff = fully_connected(layer, 64, activation='relu', weights_init='xavier')
    layer = tf.concat([layer_pc, layer_diff], axis=1)    
    layer = fully_connected(layer, 32, activation='relu', weights_init='xavier')
    logits = fully_connected(layer, n_classes, activation='linear', weights_init='xavier')
    
#     diff_dims = in_data['train'].feed[0].shape    
#     with tf.variable_scope('diff_based_net') as scope:
#         feed_pl = tf.placeholder(tf.float32, shape = (None, ) + diff_dims)
#         labels_pl = tf.placeholder(tf.int64, shape=[None])
    
#         if arch == 'mlp':
#             layer = fully_connected(feed_pl, 4, activation='relu', weights_init='xavier')
#             layer = fully_connected(layer, 6, activation='relu', weights_init='xavier')
#             logits = fully_connected(layer, n_classes, activation='linear', weights_init='xavier')

#         elif arch == 'conv':
#             layer = conv_2d(tf.expand_dims(feed_pl, -1), nb_filter=4, filter_size=4, activation='relu')
#             layer = max_pool_2d(layer, kernel_size=2)
#             layer = conv_2d(layer, nb_filter=4, filter_size=3, activation='relu')
#             layer = max_pool_2d(layer, kernel_size=2)
#             layer = fully_connected(layer, 12, activation='relu', weights_init='xavier')
#             logits = fully_connected(layer, n_classes, activation='linear', weights_init='xavier')
#         else:
#             assert(False)

print count_trainable_parameters()

18599


In [177]:
prediction = tf.argmax(logits, axis=1)
target_ = labels_pl
correct_pred = tf.equal(prediction, target_)
avg_accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
one_hot_labels = tf.one_hot(labels_pl, depth=n_classes)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_labels)
loss = tf.reduce_mean(cross_entropy)

learning_rate = 0.001
opt = tf.train.AdamOptimizer(learning_rate)
opt_step = opt.minimize(loss)

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
init=tf.global_variables_initializer()
sess.run(init)